<a href="https://colab.research.google.com/github/lydia-r/Data-projects/blob/main/Housing%20affordability%20along%20purple%20line/API%20census.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %% --------------------------------------------
# Housing Affordability (Dynamics-Driven) – ACS 5-year Block Groups
# - t0 = 2014 (2010–2014)
# - t1 = 2023 (2019–2023)
# Montgomery County, MD (state 24, county 031)
# Outputs: CSV with CHAI_t0, CHAI_t1, DCHAI and ACS-only Δ predictors
# -----------------------------------------------

import requests
import pandas as pd
import time

# ---------- CONFIG ----------
API_KEY = "8c71bf7e83ffa6cb61c568bf35ac1b1b079da9b3"  # your Census API key
state_fips = "24"
county_fips = "031"

t0_year = 2014   # ACS 5-year (2010–2014)
t1_year = 2023   # ACS 5-year (2019–2023)

out_csv = r"C:\Data\HousingAffordability\acs_bg_deltas_montgomery.csv"

# ---------- ACS variables (minimal, dependable for CHAI + core SOCIO deltas) ----------
VARS = {
    # Income
    "B19013_001E": "MEDHINC",                 # Median household income (annual $)
    # Renter cost (monthly)
    "B25064_001E": "MEDRENT",                 # Median gross rent (monthly $)
    # Owner cost (monthly) – median SMOC with mortgage
    "B25088_002E": "MED_SMOC_MORTG",          # Median SMOC (with mortgage, monthly $)
    # Tenure counts
    "B25003_002E": "OWNER_OCC",               # Owner-occupied housing units (count)
    "B25003_003E": "RENTER_OCC",              # Renter-occupied housing units (count)
    # Vacancy
    "B25002_001E": "HU_TOTAL",                # Total housing units
    "B25002_003E": "HU_VACANT",               # Vacant housing units
    # Population / Households
    "B01003_001E": "POP_TOTAL",               # Total population
    "B11001_001E": "HH_TOTAL",                # Total households
    # Education 25+ (for %BA+)
    "B15003_001E": "EDU_TOTAL_25P",           # Total population 25+
    "B15003_022E": "EDU_BA",                  # Bachelor's
    "B15003_023E": "EDU_MA",                  # Master's
    "B15003_024E": "EDU_PROF",                # Professional school
    "B15003_025E": "EDU_PHD"                  # Doctorate
}

SENTINEL_THRESHOLD = -1e8  # catches -666666666, -555555555, -222222222, etc.

# ---------- Helper functions ----------

def census_acs5_blockgroups(year:int, vars_map:dict, state:str, county:str, api_key:str, max_retries:int=5) -> pd.DataFrame:
    """Download ACS5 for all block groups in a county for specified variables."""
    base = f"https://api.census.gov/data/{year}/acs/acs5"
    var_keys = list(vars_map.keys())
    get_vars = ",".join(["NAME"] + var_keys)

    params = {
        "get": get_vars,
        "for": "block group:*",
        "in": f"state:{state} county:{county} tract:*",
        "key": api_key
    }

    for attempt in range(1, max_retries+1):
        resp = requests.get(base, params=params, timeout=60)
        if resp.status_code == 200:
            rows = resp.json()
            cols = rows[0]
            data = rows[1:]
            df = pd.DataFrame(data, columns=cols)

            # Convert requested variables to numeric and null-out sentinel codes
            for v in var_keys:
                df[v] = pd.to_numeric(df[v], errors="coerce")
                df.loc[df[v] < SENTINEL_THRESHOLD, v] = pd.NA

            # GEOID and friendly names
            df["GEOID"] = df["state"] + df["county"] + df["tract"] + df["block group"]
            df = df.rename(columns={k: vars_map[k] for k in vars_map})

            keep = ["GEOID","state","county","tract","block group","NAME"] + list(vars_map.values())
            return df[keep]

        else:
            wait = 2 ** (attempt - 1)
            print(f"[{year}] Census API HTTP {resp.status_code} – retrying in {wait}s...")
            try:
                print("Message:", resp.json())
            except Exception:
                pass
            time.sleep(wait)

    raise RuntimeError(f"Failed to download ACS {year} after {max_retries} attempts.")

def build_chai(df: pd.DataFrame, suffix:str) -> pd.DataFrame:
    """
    Compute tenure-weighted Composite Housing Affordability Index (CHAI) for a given time suffix.
    Handles ACS sentinel values and tenure=0 cases.

    CHAI_t = (RenterShare * RB) + (OwnerShare * OB),
    RB = (MEDRENT * 12) / MEDHINC,
    OB = (MED_SMOC_MORTG * 12) / MEDHINC
    """
    inc    = f"MEDHINC{suffix}"
    rent   = f"MEDRENT{suffix}"
    smoc   = f"MED_SMOC_MORTG{suffix}"
    owner  = f"OWNER_OCC{suffix}"
    renter = f"RENTER_OCC{suffix}"

    # Ensure numeric & sentinel -> NaN
    for col in (inc, rent, smoc, owner, renter):
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df.loc[df[col] < SENTINEL_THRESHOLD, col] = pd.NA

    # Household counts and tenure shares
    hh = df[[owner, renter]].sum(axis=1)
    renter_share = df[renter] / hh.replace({0: pd.NA})
    owner_share  = df[owner]  / hh.replace({0: pd.NA})

    # Burdens (as fractions of income)
    rb = (df[rent] * 12.0) / df[inc]   # renter burden
    ob = (df[smoc] * 12.0) / df[inc]   # owner burden

    # CHAI logic: only renters, only owners, or weighted mix
    chai = pd.Series(pd.NA, index=df.index, dtype="float64")
    only_renters = (owner_share.fillna(0) == 0) & (renter_share > 0)
    only_owners  = (renter_share.fillna(0) == 0) & (owner_share > 0)
    both         = (owner_share > 0) & (renter_share > 0)

    chai[only_renters] = rb[only_renters]
    chai[only_owners]  = ob[only_owners]
    chai[both]         = (renter_share[both] * rb[both]) + (owner_share[both] * ob[both])

    df[f"CHAI{suffix}"] = chai
    df[f"RenterShare{suffix}"] = renter_share
    df[f"OwnerShare{suffix}"]  = owner_share
    df[f"RB{suffix}"]          = rb
    df[f"OB{suffix}"]          = ob
    return df

def compute_basic_deltas(df_t0: pd.DataFrame, df_t1: pd.DataFrame) -> pd.DataFrame:
    """Merge t0/t1 frames on GEOID and compute Δ for ACS-only predictors & CHAI."""
    df0 = df_t0.add_suffix("_t0")
    df1 = df_t1.add_suffix("_t1")

    # Restore ID columns
    for c in ["GEOID","state","county","tract","block group","NAME"]:
        if c+"_t0" in df0.columns: df0 = df0.rename(columns={c+"_t0": c})
        if c+"_t1" in df1.columns: df1 = df1.rename(columns={c+"_t1": c})

    on_cols = ["GEOID","state","county","tract","block group"]
    merged = df0.merge(df1, on=on_cols, how="inner", suffixes=("_t0","_t1"))

    # Build CHAI at each timepoint (handles sentinel & tenure-zero)
    merged = build_chai(merged, "_t0")
    merged = build_chai(merged, "_t1")

    # Δ dependent variable
    merged["DCHAI"] = merged["CHAI_t1"] - merged["CHAI_t0"]

    # Δ predictors (ACS-only)
    def dcol(base):
        merged[f"d_{base}"] = pd.to_numeric(merged[f"{base}_t1"], errors="coerce") - pd.to_numeric(merged[f"{base}_t0"], errors="coerce")

    for base in ["MEDHINC","POP_TOTAL","HH_TOTAL"]:
        dcol(base)

    # Vacancy rate Δ
    merged["VacancyRate_t0"] = merged["HU_VACANT_t0"] / merged["HU_TOTAL_t0"]
    merged["VacancyRate_t1"] = merged["HU_VACANT_t1"] / merged["HU_TOTAL_t1"]
    merged["d_VacancyRate"]  = merged["VacancyRate_t1"] - merged["VacancyRate_t0"]

    # % BA+ Δ
    merged["BAplus_t0"] = (
        (merged["EDU_BA_t0"] + merged["EDU_MA_t0"] + merged["EDU_PROF_t0"] + merged["EDU_PHD_t0"])
        / merged["EDU_TOTAL_25P_t0"]
    )
    merged["BAplus_t1"] = (
        (merged["EDU_BA_t1"] + merged["EDU_MA_t1"] + merged["EDU_PROF_t1"] + merged["EDU_PHD_t1"])
        / merged["EDU_TOTAL_25P_t1"]
    )
    merged["d_PctBAplus"] = merged["BAplus_t1"] - merged["BAplus_t0"]

    # Year stamps
    merged.insert(1, "t0_year", t0_year)
    merged.insert(2, "t1_year", t1_year)

    keep_cols = [
        "GEOID","t0_year","t1_year","state","county","tract","block group",
        "NAME_t0","NAME_t1",
        "CHAI_t0","CHAI_t1","DCHAI",
        "RenterShare_t0","OwnerShare_t0","RB_t0","OB_t0",
        "RenterShare_t1","OwnerShare_t1","RB_t1","OB_t1",
        "d_MEDHINC","d_PctBAplus","d_VacancyRate","d_POP_TOTAL","d_HH_TOTAL",
        "MEDHINC_t0","MEDHINC_t1","MEDRENT_t0","MEDRENT_t1",
        "MED_SMOC_MORTG_t0","MED_SMOC_MORTG_t1",
        "POP_TOTAL_t0","POP_TOTAL_t1","HH_TOTAL_t0","HH_TOTAL_t1",
        "HU_TOTAL_t0","HU_TOTAL_t1","HU_VACANT_t0","HU_VACANT_t1",
        "EDU_TOTAL_25P_t0","EDU_TOTAL_25P_t1",
        "EDU_BA_t0","EDU_MA_t0","EDU_PROF_t0","EDU_PHD_t0",
        "EDU_BA_t1","EDU_MA_t1","EDU_PROF_t1","EDU_PHD_t1"
    ]
    keep_cols = [c for c in keep_cols if c in merged.columns]
    return merged[keep_cols].copy()

# ---------- Run ----------
print(f"Downloading ACS {t0_year} 5-year (t0)...")
acs_t0 = census_acs5_blockgroups(t0_year, VARS, state_fips, county_fips, API_KEY)
print(f"t0 rows: {len(acs_t0)}")

print(f"Downloading ACS {t1_year} 5-year (t1)...")
acs_t1 = census_acs5_blockgroups(t1_year, VARS, state_fips, county_fips, API_KEY)
print(f"t1 rows: {len(acs_t1)}")

print("Merging & computing CHAI and deltas...")
df_out = compute_basic_deltas(acs_t0, acs_t1)

print(f"Writing CSV → {out_csv}")
df_out.to_csv(out_csv, index=False)
print("DONE ✔")